In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import xlrd

In [2]:
df_og = pd.read_excel('cross-border_alldeals.xlsx', index_col=0)

In [3]:
df = df_og.copy()
df = df.reset_index()
df.head()

,Deal Number,Acquiror name,Acquiror country code,Target name,Target country code,Deal type,Deal status,Last deal status date,Deal value th EUR,Target GUO Name,Target GUO ISO country code,Acquiror GUO Name,Acquiror GUO ISO country code,Target primary NACE Rev.2 code
0,1.909154e+09,SENSORMATIC ELECTRONICS CORPORATION,US,PLASTROM SENSORMATIC,BR,Joint venture 100%,Completed,1991-12-31,n.a.,NaN,NaN,JOHNSON CONTROLS INTERNATIONAL PLC,IE,1729.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,1.909194e+09,CUMMINS INC.,US,"CHONGQING CUMMINS ENGINE CO., LTD",CN,Joint venture 100%,Completed,1995-12-31,n.a.,NaN,NaN,CUMMINS INC.,US,2811.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,CHONGQING MACHINERY & ELECTRONICS HOLDING (GRO...,CN,NaN
4,1.909090e+09,PEDER SMEDVIG CAPITAL LTD,GB,PETROLERA EL TREBOL SA,AR,Minority stake unknown %,Completed,1996-01-01,3071.27,NaN,NaN,SOTEIRA LTD. AS TRUSTEE FOR SOPHIA T,NO,610.0


In [4]:
#Change n.a. to NaN
df = df.replace('n.a.', np.nan)
df = df.replace('-', np.nan)

In [5]:
#Important columns: Deal Number, Acquiror country code, Target country code, Deal status, Last deal status date,
#Target GUO ISO country code, Target GUO ISO country code

#Remove unnecessary columns
cols_todrop = ['Acquiror name', 'Target name', 'Deal type', 'Target GUO Name', 
               'Acquiror GUO Name', 'Deal value\nth EUR']
df = df.drop(columns=cols_todrop,axis=1)
df.head()

,Deal Number,Acquiror country code,Target country code,Deal status,Last deal status date,Target GUO ISO country code,Acquiror GUO ISO country code,Target primary NACE Rev.2 code
0,1.909154e+09,US,BR,Completed,1991-12-31,NaN,IE,1729.0
1,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,1.909194e+09,US,CN,Completed,1995-12-31,NaN,US,2811.0
3,NaN,NaN,NaN,NaN,NaT,NaN,CN,NaN
4,1.909090e+09,GB,AR,Completed,1996-01-01,NaN,NO,610.0


In [6]:
#Remove every row with no deal number (Gets rid of the secondary acquirors)
df = df.dropna(subset=df.columns[[0]], how='any')
df = df.reset_index()
df.head()

,index,Deal Number,Acquiror country code,Target country code,Deal status,Last deal status date,Target GUO ISO country code,Acquiror GUO ISO country code,Target primary NACE Rev.2 code
0,0,1.909154e+09,US,BR,Completed,1991-12-31,NaN,IE,1729.0
1,2,1.909194e+09,US,CN,Completed,1995-12-31,NaN,US,2811.0
2,4,1.909090e+09,GB,AR,Completed,1996-01-01,NaN,NO,610.0
3,5,1.633059e+09,JP,VG,Completed,1996-06-24,NaN,JP,2351.0
4,6,1.592150e+05,SE,JP,Completed,1996-12-19,NaN,SE,7021.0


In [7]:
#Drop deal number
df = df.drop(columns=['Deal Number'],axis=1)

In [8]:
#Remove deals that aren't completed (This removes about 90,000 rows)
df = df[df['Deal status'] == 'Completed']
df = df.reset_index()

In [9]:
#Next, consolidate Acquiror/Target country codes with GUO ISO country code.
#If the GUO ISO country code exists for the row, I change the 'Acquiror country code' and 'Target country code'
#columns to match. Both GUO ISO columns will be deleted after this. 
df['Target country code'] = np.where(~df['Target GUO ISO country code'].isna(), 
                                     df['Target GUO ISO country code'], df['Target country code'])

df['Acquiror country code'] = np.where(~df['Acquiror GUO ISO country code'].isna(), 
                                     df['Acquiror GUO ISO country code'], df['Acquiror country code'])
dropped_cols = ['Target GUO ISO country code', 'Acquiror GUO ISO country code', 'level_0', 'index']
df = df.drop(columns=dropped_cols,axis=1)
#Deleting 2 extra columns in front as well 

In [10]:
df.head(10)


,Acquiror country code,Target country code,Deal status,Last deal status date,Target primary NACE Rev.2 code
0,IE,BR,Completed,1991-12-31,1729.0
1,US,CN,Completed,1995-12-31,2811.0
2,NO,AR,Completed,1996-01-01,610.0
3,JP,VG,Completed,1996-06-24,2351.0
4,SE,JP,Completed,1996-12-19,7021.0
5,BR,BR,Completed,1997-01-01,1013.0
6,GB,AU,Completed,1997-01-01,5829.0
7,US,FI,Completed,1997-01-02,6201.0
8,DE,US,Completed,1997-01-02,2014.0
9,GB,SE,Completed,1997-01-02,2561.0


In [11]:
#Rename 'Target primary NACE Rev.2 code' to something more intuitive
df = df.rename(index=str, columns={"Target primary NACE Rev.2 code": "Industry code"})

In [12]:
#Pull only year from dates
df['year']=pd.to_datetime(df['Last deal status date'], format='%y/%m/%d').dt.year.fillna(0.0).astype(int)
df = df.drop(columns=['Last deal status date'],axis=1)

In [13]:
df.head()

,Acquiror country code,Target country code,Deal status,Industry code,year
0,IE,BR,Completed,1729.0,1991
1,US,CN,Completed,2811.0,1995
2,NO,AR,Completed,610.0,1996
3,JP,VG,Completed,2351.0,1996
4,SE,JP,Completed,7021.0,1996


In [14]:
#Remove NaNs from the industry codes
df = df.dropna(subset=df.columns[[3]], how='any')
df = df.reset_index()
df = df.drop(columns=['index'],axis=1)
df.head()

,Acquiror country code,Target country code,Deal status,Industry code,year
0,IE,BR,Completed,1729.0,1991
1,US,CN,Completed,2811.0,1995
2,NO,AR,Completed,610.0,1996
3,JP,VG,Completed,2351.0,1996
4,SE,JP,Completed,7021.0,1996


In [15]:
# Still NaNs in Acquiror and Target; data is just missing so these rows will be removed. ~7000 rows of data (marginal)
df = df.dropna(subset=df.columns[[0,1]], how='any')
df = df.reset_index()
df = df.drop(columns=['index'],axis=1)
df.head()

,Acquiror country code,Target country code,Deal status,Industry code,year
0,IE,BR,Completed,1729.0,1991
1,US,CN,Completed,2811.0,1995
2,NO,AR,Completed,610.0,1996
3,JP,VG,Completed,2351.0,1996
4,SE,JP,Completed,7021.0,1996


In [16]:
# To make counting easier, going to replace all of the numerical industry codes with their written counterpart using a functor
def convertToName(x):
    if (x > 99 and x < 400):
        return 'Primary'
    elif (x > 499 and x < 1000):
        return 'Mining and Quarrying'
    elif (x > 999 and x < 1900):
        return 'Manufacturing of Food, Beverages, Textiles, Wood, and Papers'
    elif (x > 1899 and x < 2300):
        return 'Manufacturing of Oil and Chemicals (includes pharmaceuticals)'
    elif (x > 2299 and x < 2600):
        return 'Manufacturing of Metals'
    elif (x > 2599 and x < 2900):
        return 'Manufacturing of Electronic goods and Machinery'
    elif (x > 2899 and x < 3100):
        return 'Manufacturing of Transport Equipment'
    elif (x > 3499 and x < 3600):
        return 'Electricity'
    elif (x > 4099 and x < 4400):
        return 'Construction'
    elif (x > 4499 and x < 4800):
        return 'Distribution (Wholesale and Retail Trade)'
    elif (x > 4899 and x < 5400):
        return 'Transportation (includes land, maritime, air, cargo, postal activities)'
    elif (x > 5499 and x < 5700):
        return 'Hotels and Restaurants'
    elif (x > 5799 and x < 6100):
        return 'Media (includes newspapers, magazines, and TV broadcasting)'
    elif (x > 6099 and x < 6200):
        return 'Telecommunications'
    elif (x > 6199 and x < 6400):
        return 'Computers and Information'
    elif (x > 6399 and x < 6700):
        return 'Financial and Insurance'
    elif (x > 6799 and x < 6900):
        return 'Real Estate'
    elif (x > 6899 and x < 7300):
        return 'Professional Activities (accounting, architectural, legal, engineering,scientific research and development)'
    elif ((x > 7299 and x < 8300) or (x > 9399 and x < 9700)):
        return 'Other Business Services'
    elif (x > 8399 and x < 8500):
        return 'Public Administration and Defense'
    elif (x > 8499 and x < 8900):
        return 'Education and Social Services'
    elif(x > 8999 and x < 9400):
        return 'Recreation and Entertainment'

In [17]:
#Change codes to names
df['Industry code'] = df['Industry code'].apply(convertToName)
df.head()

,Acquiror country code,Target country code,Deal status,Industry code,year
0,IE,BR,Completed,"Manufacturing of Food, Beverages, Textiles, Wo...",1991
1,US,CN,Completed,Manufacturing of Electronic goods and Machinery,1995
2,NO,AR,Completed,Mining and Quarrying,1996
3,JP,VG,Completed,Manufacturing of Metals,1996
4,SE,JP,Completed,"Professional Activities (accounting, architect...",1996


In [18]:
#table = pivot_table(df, values='D', index=['A', 'B'],
#...                     columns=['C'], aggfunc=np.sum, fill_value=0)
table = pd.pivot_table(df, values='Industry code', index=['Acquiror country code', 'Target country code', 'year', 'Industry code'],  aggfunc=np.count_nonzero)
table

Deal status
Acquiror country code Target country code year Industry code                                                  
AD                    AD                  2006 Manufacturing of Food, Beverages, Textiles, Woo...            1
                                          2012 Financial and Insurance                                       1
                                          2015 Financial and Insurance                                       1
                      CH                  2011 Financial and Insurance                                       1
                                          2013 Financial and Insurance                                       1
                      ES                  2005 Financial and Insurance                                       1
                                          2007 Financial and Insurance                                       1
                                          2008 Financial and Insurance                                       1
                                          2009 Financial and Insurance                                       1
                                               Recreation and Entertainment                                  1
                                          2011 Financial and Insurance                                       2
                                          2012 Financial and Insurance                                       1
                                          2013 Distribution (Wholesale and Retail Trade)                     1
                                          2015 Financial and Insurance                                       1
                                          2016 Hotels and Restaurants                                        1
                                          2017 Hotels and Restaurants                                        1
                      GR                  2009 Primary                                                       1
                      MX                  2008 Financial and Insurance                                       1
                      MY                  2013 Manufacturing of Oil and Chemicals (includes ph...            1
                      PE                  2012 Financial and Insurance                                       1
                      RU                  2008 Financial and Insurance                                       1
                                               Manufacturing of Electronic goods and Machinery               1
                      VN                  2014 Transportation (includes land, maritime, air, c...            2
                                          2015 Mining and Quarrying                                          1
                                               Transportation (includes land, maritime, air, c...            1
                                          2016 Manufacturing of Metals                                       2
                                          2017 Manufacturing of Metals                                       1
AE                    AE                  1999 Manufacturing of Food, Beverages, Textiles, Woo...            1
                                          2003 Construction                                                  1
                                               Professional Activities (accounting, architectu...            1
...                                                                                                        ...
ZW                    BW                  2005 Telecommunications                                            1
                                          2013 Financial and Insurance                                       1
                      CD                  2004 Hotels and Restaurants                                        1
                      CH                  2016 Telecommunications                                      

In [20]:
# Export final count
table.to_csv('Industry_Count.csv')